In [1]:
import os

from pydrake.common import temp_directory

In [2]:
# Create a Drake temporary directory to store files.
# Note: this tutorial will create a temporary file (table_top.sdf)
# in the `/tmp/robotlocomotion_drake_xxxxxx` directory.
temp_dir = temp_directory()

# Create a table top SDFormat model.
table_top_sdf_file = os.path.join(temp_dir, "table_top.sdf")
table_top_sdf = """<?xml version="1.0"?>
<sdf version="1.7">

  <model name="table_top">
    <link name="table_top_link">
      <inertial>
        <mass>18.70</mass>
        <inertia>
          <ixx>0.79</ixx>
          <ixy>0</ixy>
          <ixz>0</ixz>
          <iyy>0.53</iyy>
          <iyz>0</iyz>
          <izz>1.2</izz>
        </inertia>
      </inertial>
    <visual name="bottom">
        <pose>0.0 0.0 0.445 0 0 0</pose>
        <geometry>
          <box>
            <size>0.49 0.63 0.015</size>
          </box>
        </geometry>
        <material>
          <diffuse>0.9 0.9 0.9 1.0</diffuse>
        </material>
      </visual>
      <collision name="bottom">
        <pose>0.0 0.0 0.445 0 0 0</pose>
        <geometry>
          <box>
            <size>0.49 0.63 0.015</size>
          </box>
        </geometry>
        <drake:proximity_properties>
          <drake:compliant_hydroelastic/>
          <drake:hydroelastic_modulus>1.0e6</drake:hydroelastic_modulus>
        </drake:proximity_properties>
      </collision>
    </link>
    <frame name="table_top_center">
      <pose relative_to="table_top_link">0 0 0.47 0 0 0</pose>
    </frame>
  </model>
</sdf>

"""

with open(table_top_sdf_file, "w") as f:
    f.write(table_top_sdf)

In [3]:
cwd = os.getcwd()
your_model_filename = cwd+"/Board.sdf" # Write the absolute path to your file here

In [4]:
def convert_to_coords(pos):
    x = ord(pos[0]) - ord('a')
    y = int(pos[1])-1
    return "" + str(.6*.125*(y-3.5)) + ", "+ str(.6*.125*(x-3.5)+0.55) 

def get_coord_from_pos(pos):
    x = ord(pos[0]) - ord('a')
    y = int(pos[1])-1
    return [.6*.125*(y-3.5),.6*.125*(x-3.5)+0.55]

def add_piece(piece_id, name,  pos):
    return f"""
- add_model:
    name: {piece_id}
    file: file://{cwd}/{name}.sdf
    default_free_body_pose:
        {name}:
            translation: [{convert_to_coords(pos)}, .012]
            rotation: !Rpy {{ deg: [0, 0, 0] }}"""

In [5]:
scenario = None
def create_scene():
    global scenario
    scenario_data = f"""
directives:
- add_model:
    name: table_top
    file: file://{table_top_sdf_file}
- add_weld:
    parent: world
    child: table_top::table_top_center
"""
    scenario_data += """
    X_PC:
        translation: [0, 0.55, 0]
        rotation: !Rpy { deg: [0, 0, 0]}
"""
    if your_model_filename:
        scenario_data += f"""
- add_model:
    name: board
    file: file://{your_model_filename}
    default_free_body_pose:
        Board:
            translation: [0, 0.55, .01]
            rotation: !Rpy {{ deg: [90, 0, 0] }}    
"""
    scenario_data += add_piece('p1', 'BlackKnight',  'b8')
    scenario_data += add_piece('p2', 'BlackBishop',  'c8')
    scenario_data += add_piece('p3', 'BlackKnight',  'g8')
    scenario_data += add_piece('p4', 'BlackBishop',  'f8')
    scenario_data += add_piece('p5', 'BlackQueen',  'd8')
    scenario_data += add_piece('p6', 'BlackRook',  'a8')
    scenario_data += add_piece('p7', 'BlackRook',  'h8')
    scenario_data += add_piece('p8', 'BlackKing',  'e8') 
    scenario_data += add_piece('p9', 'WhiteKnight',  'b1')
    scenario_data += add_piece('p10', 'WhiteBishop',  'c1')
    scenario_data += add_piece('p11', 'WhiteKnight',  'g1')
    scenario_data += add_piece('p12', 'WhiteBishop',  'f1')
    scenario_data += add_piece('p13', 'WhiteQueen',  'd1')
    scenario_data += add_piece('p14', 'WhiteRook',  'a1')
    scenario_data += add_piece('p15', 'WhiteRook',  'h1')
    scenario_data += add_piece('p16', 'WhiteKing',  'e1')
    for i in range(8): 
       scenario_data += add_piece(f"p{17+i}", 'WhitePawn',  f"{chr(97+i)}2")
       scenario_data += add_piece(f"p{25+i}", 'BlackPawn',  f"{chr(97+i)}7")
    #scenario_data += add_piece('p1', 'WhitePawn',  'e2')
    
    scenario_data += """
- add_model:
    name: iiwa
    file: package://drake/manipulation/models/iiwa_description/sdf/iiwa14_no_collision.sdf
    default_joint_positions:
        iiwa_joint_1: [1.57]
        iiwa_joint_2: [0.1]
        iiwa_joint_3: [0]
        iiwa_joint_4: [-1.2]
        iiwa_joint_5: [0]
        iiwa_joint_6: [ 1.6]
        iiwa_joint_7: [0]
- add_weld:
    parent: world
    child: iiwa::iiwa_link_0

- add_model:
    name: wsg
    file: package://manipulation/hydro/schunk_wsg_50_with_tip.sdf
- add_weld:
    parent: iiwa::iiwa_link_7
    child: wsg::body
    X_PC:
        translation: [0, 0, 0.09]
        rotation: !Rpy { deg: [90, 0, 90]}
- add_model:
    name: camera
    file: package://manipulation/camera_box.sdf
- add_weld:
    parent: wsg::body
    child: camera::base
    X_PC:
        translation: [0, 0.03, 0.04]
        # Point slightly down towards camera
        # RollPitchYaw(0, -0.2, 0.2) @ RollPitchYaw(-np.pi/2, 0, np.pi/2)
        rotation: !Rpy { deg: [-90, 0, 0] }
model_drivers:
    iiwa: !IiwaDriver
        hand_model_name: wsg
    wsg: !SchunkWsgDriver {}
cameras:
    main_camera:
        name: camera0
        depth: True
        X_PB:
            base_frame: camera::base
"""
    with open('board.yaml', 'w') as f:
        f.write(scenario_data)

    return scenario_data

In [6]:
create_scene()

'\ndirectives:\n- add_model:\n    name: table_top\n    file: file:///tmp/robotlocomotion_drake_jdL2nB/table_top.sdf\n- add_weld:\n    parent: world\n    child: table_top::table_top_center\n\n    X_PC:\n        translation: [0, 0.55, 0]\n        rotation: !Rpy { deg: [0, 0, 0]}\n\n- add_model:\n    name: board\n    file: file:///workspaces/robot/Board.sdf\n    default_free_body_pose:\n        Board:\n            translation: [0, 0.55, .01]\n            rotation: !Rpy { deg: [90, 0, 0] }    \n\n- add_model:\n    name: p1\n    file: file:///workspaces/robot/BlackKnight.sdf\n    default_free_body_pose:\n        BlackKnight:\n            translation: [0.2625, 0.36250000000000004, .012]\n            rotation: !Rpy { deg: [0, 0, 0] }\n- add_model:\n    name: p2\n    file: file:///workspaces/robot/BlackBishop.sdf\n    default_free_body_pose:\n        BlackBishop:\n            translation: [0.2625, 0.43750000000000006, .012]\n            rotation: !Rpy { deg: [0, 0, 0] }\n- add_model:\n    name

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=c8432d35-2370-4de9-924c-af3802cf315d' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>